# Cost of Plutocracy

The aim of this article is to analyze some of the biggest DAOs in the crypto space and see how many of them are influenced by token holders which own a disproportionate amount of tokens compared to most voters.

In [1]:
# sets up the pynb environment
import os
import sys

import pandas as pd
import numpy as np


module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from libs.data_processing.filters import get_quartile_by_vp
from libs.data_processing.statistics import (
    get_number_of_whales_to_all_voters_ratio,
    get_score_comparisons,
)


The data used for this article came from data sources that specialize in providing data on DAO governance, such as DeepDAO and Snapshot. Coingecko was also used to find market data for DAO native tokens.

Using this data, two spreadsheets were compiled which act as the local database for this analysis. Each spreadsheet contains each voter's choice and voting power for the last five proposals each DAO opened. **One spreadsheet filters out "whales"** which, in the context of this analysis, are voters of each proposal which have voting power **at or above the 95th percentile of voting power for that proposal**.

In [2]:
plutocracy_report_data = pd.read_excel(
    "../plutocracy_report.xls", sheet_name=None, engine="openpyxl"
)
plutocracy_report_data_filtered = pd.read_excel(
    "../plutocracy_report_filtered.xls", sheet_name=None, engine="openpyxl"
)


With this, we can begin asking ourselves some questions. Such as: **How many whales have voted in the last five proposals for each of the DAOs we analyzed?**

In [3]:
voting_ratios = get_number_of_whales_to_all_voters_ratio(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [4]:
pd.DataFrame(
    [list(result.items())[0][1] for result in voting_ratios],
    index=[list(result.items())[0][0] for result in voting_ratios],
    columns=[
        "# of whales",
        "all voters",
        "avg vp for non-whales",
        "avg vp for whales",
        "avg cost of vote",
    ],
)


,# of whales,all voters,avg vp for non-whales,avg vp for whales,avg cost of vote
Uniswap,568,12250,1.402462e+00,8.086719e+04,6.092379
ENS,74,1609,3.521251e+00,3.356937e+04,14.895998
Lido,129,2396,2.996389e+00,7.571993e+05,1.455704
Frax,1,53,9.872141e+04,2.893798e+06,10.223617
Decentraland,18,352,6.129250e+01,1.409692e+05,0.753464
Curve Finance,8,181,6.146095e+03,2.509741e+06,2.485927
Radicle,1,28,2.130899e+05,2.830379e+06,4.079941
Euler,3,81,1.014071e+03,6.796260e+04,5.253017
JPEG’d,3,87,4.154734e+07,7.201127e+08,0.000875


Here we can clearly see that for the DAOs which we choose to analyze the top 5% of voters have a clear, disproportionate amount of voting power comparared to the average voter.

We can also gain some insight at the economic might that these whales hold for each DAO at the time of voting by taking a look at the average cost of each vote for each DAO's native token at the time these proposals were active.

---

We move on to taking at what DAO participation would look like without these whale's involvement. We do so by comparing the scores of each proposal and checking whether or not the outcome of the proposal was changed after filtering out whales.

We check whether or not a propsal's outcome has changed simply by checking if the largest vote choice score has changed after filtering out whales, more specifically, in python we do:
```python
has_changed_outcome = not unfiltered_winning_choice_index == filtered_winning_choice_index
```

In [5]:
score_diffrences = get_score_comparisons(
    plutocracy_report_data, plutocracy_report_data_filtered
)


In [6]:
score_diffrences_dfs = dict()

for score_diffrence in score_diffrences:
    for organization, data in score_diffrence.items():
        data: dict[str, list] = data
        items = data.items()
        score_diffrences_dfs[organization] = pd.DataFrame(
            [score_data for _, score_data in items],
            index=pd.Index(
                ([proposal_id for proposal_id, _ in items]), name="Proposal ID"
            ),
            columns=[
                "type",
                "choices",
                "score_diffrences",
                "total_vp",
                "has_changed_outcome",
                "changed_outcome_winner"
            ],
        )
        score_diffrences_dfs[organization]["total_vp"] = score_diffrences_dfs[
            organization
        ]["total_vp"].apply("{:.9f}".format)


First, let's look at Uniswap

In [7]:
score_diffrences_dfs["Uniswap"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852,single-choice,"['Wormhole', 'LayerZero', 'Celer', 'deBridge']","[23275464.75822096, 17163633.19927637, 481.545...",45581689.082320422,False,Wormhole
0xa3c7247632d3a08e4e73a63867908d0cd4402f48fa673945f93e3197dee5450a,single-choice,"['Yes', 'No']","[20056524.265649345, 4927056.974342918]",24994560.158377878,False,Yes
0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c,single-choice,"['add Overnight Finance', 'add DeFiEdge', 'add...","[10054.585075893452, 2708025.0728700403, 3412....",2728492.255083876,True,add Overnight Finance
0x387b5e811e9673359b98729af5c7f14a50b7d0ea9b4dcf9278686b238932e25d,single-choice,"['Yes - Enact changes', 'No - No change']","[30508464.50667036, 34.17723847808492]",61343398.710826278,False,Yes - Enact changes


We observe that out of the last five proposals only one had their outcome changed after removing whales: The proposal to [add a Liquidity Mining Manager for the Optimism-Uniswap Liquidty Mining Program](https://snapshot.org/#/uniswap/proposal/0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c).

In this case, "Overnight Finance" would've been the chosen LMM if it weren't for whale interevention, instead "DeFiEdge" was chosen for this campaign. From the data, we can see that without whale intervention, DeFiEdge would've had ~2,708,025 less votes out of the total ~2,728,492 voting power on this proposal. That is ~99.3% of voting power allocated for this proposal directed to choosing "DeFiEdge".

In [8]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0xfd3d3807bd2a6eda1327c311b83de235061d39ff1bdfb616c9f9b0d367c3ac2c'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,add Overnight Finance,add DeFiEdge,add none
Scores,14335.930887,2.710600e+06,3556.089173
Score Differeces,10054.585076,2.708025e+06,3412.851497


One highly contentious proposal whose outcome did not change after filtering out whales was [this temperature check to choose which Eth <> BNB bridge to use for Uniswap v3 governance](https://snapshot.org/#/uniswap/proposal/0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852). We see that even for the least popular choices, whales still contributed to the majority of the votes. We also observe that whales held the overwhelming majority of voting power for the runner up, "LayerZero", with 99.9% of voting power allocated to this proposal being directed to this choice. By comparison, "Wormhole" received 65.1% of voting power for this proposal coming from whales.

In [9]:
propsal_choices = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Uniswap"].loc["0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Uniswap'][plutocracy_report_data['Uniswap']['proposal_id'] == '0x6b8df360fdf73085b21fdf5eef9f85916fbde95621a3d454cb20fbe545ffc852'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Wormhole,LayerZero,Celer,deBridge
Scores,2.840391e+07,1.717334e+07,662.606204,3772.802474
Score Differeces,2.327546e+07,1.716363e+07,481.545728,3402.436704


Next up is ENS.

In [10]:
score_diffrences_dfs["ENS"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7,single-choice,"['For', 'Against', 'Abstain']","[1667210.963067633, 0.0, 137.12454090065714]",1670185.073618348,False,For
0xd7eff781be059513b5cd64d79e709abbbc653944c9a8c621dc051e7b42a405cb,single-choice,"['For', 'Against', 'Abstain']","[1666938.9315321625, 100.38682608595627, 141.9...",1670154.219044510,False,For
0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b,single-choice,"['For', 'Against', 'Abstain']","[1446509.1013934365, 208348.53159207577, 13519...",1671687.088797375,False,For
0xdaff050d30c77fe469da7eaa5b5bae0cb892f91a89c119367e47faf370667240,approval,"['nick.eth', 'james.eth', 'accessor.eth', 'sim...","[2319757.8152544936, 2319757.8152544936, 23197...",2325021.309782755,False,nick.eth
0xa245dc7264ae072620434996e1c66d82c64cefc5a2edde5114d691a8559d0b5f,approval,"['thecap.eth (Cap)', 'slobo.eth', 'limes.eth',...","[2278346.061608189, 2278346.0616081893, 227834...",2282885.783012460,False,slobo.eth


We see that none of the proposal outcomes were changed after filtering out whales. However, if we take look at score differences as they compare to the score differences, we see that they were almost all determined by whales.

In [11]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.669865e+06,125.702762,194.817417
Score Differeces,1.667211e+06,0.000000,137.124541


[This proposal](https://snapshot.org/#/ens.eth/proposal/0x41b3509b88e15677aa15680f48278517f794822fb9a79b9c621def53f1866be7) once passed will approve the funding of various grants ENS has commited to funding as detailed by the proposal, **the amount of capital to be commited to this proposal is 250k USDC and 50 ETH**. 99.9% of voting power allocated towards this propsal was commited to voting for its passing, 0.0001% was allocated to abstaining and even a smaller percentage was allocated to voting against its passing.

Meaning that ~0.1% of the vote would've still been allocated towards voting for the proposal to pass even without whale intervention, which I think many would see as fine since funding of public goods is always important.

Another propsal on ENS which had a large amount of voting power committed by whales is [this proposal](https://snapshot.org/#/ens.eth/proposal/0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b) to commit **935k USDC and 254 ETH** to the ENS Ecosystem Working group which is responsible for *growing and improving the ENS Ecosystem by funding people and projects that are ENS-specific or ENS-centric*.

In [12]:
propsal_choices = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["ENS"].loc["0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b"]["score_diffrences"]
proposal_scores = plutocracy_report_data['ENS'][plutocracy_report_data['ENS']['proposal_id'] == '0x5788bf0f52ce82a1d3f7750a80f3001671ded49e4e0239dbbafd154275c78f8b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against,Abstain
Scores,1.449464e+06,208582.378002,13640.353424
Score Differeces,1.446509e+06,208348.531592,13519.449465


Similar to the previous proposal we looked at, this one was highly influenced by whales (~86.5% of voting power committed to this proposal coming from whales that voted for its passing, ~12.5% from whales which voted against it, and ~0.8% of whales voted to abstain).

Yet, even after removing whales, the proposal still passes. After doing some quick math, we can see the the remaining voting power for this proposal that voted for its passing equates to about ~0.17% out of the remaining 0.2% of voting power after removing whales.

Still, I am not at all saying that the outcome of this proposal is controversial. An organisation funding initiatives that aid in the development of its product should not be a controversial topic for its members. I suppose the only thing that would be contested is the amount that is committed to this proposal.

Next lets take a look at everyone's favourite liquid staking platform: Lido.

In [13]:
score_diffrences_dfs["Lido"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x3436b98afacec300ec2c0f79bb8dfd16685ececeeb7a45a18754d43b710179e8,single-choice,"['For', 'Against']","[55124323.664074376, 2048432.8880345668]",57176658.163439780,False,For
0x32f6f095dc4f7b97665b560781d6e6795da11f9c8218936a505575040038d70f,single-choice,"['For', 'Against']","[57163748.21209293, 7563.113010643607]",57175091.606986538,False,For
0x7ac2431dc0eddcad4a02ba220a19f451ab6b064a0eaef961ed386dc573722a7f,single-choice,"['For', 'Against']","[57686947.46411716, 78.13038422652139]",57690201.588526487,False,For
0x7f19fed19b66df88b45d64c22f9905fbf716839a5052fb79b10ee2ef1241645d,single-choice,"['YAY', 'NAY']","[54546195.85193226, 7525.505910950667]",54557517.318199627,False,YAY
0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b,single-choice,"['1. For 6 new oracles, 6/11 set', '2. For 4 n...","[92934.92264371048, 52723122.2034864, 4535.772...",52826368.429703690,True,"1. For 6 new oracles, 6/11 set"


Here, we see that there was a change in outcome for one of the proposals. [This proposal](https://snapshot.org/#/lido-snapshot.eth/proposal/0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b) to [expand the oracle set and quorum](https://mainnet.lido.fi/#/lido-dao/0x442af784a788a5bd6f42a01ebe9f287a871243fb/) of Lido DAO oracle node operators was passed 16/12/22 and onboarded 4 new oracle node operators and set the quorum to 5/9 reports from orales being identical to be accepted by the protocol.

Had whales not intervened, Option 1 to onboard 6 additional oracles and set the quorum to 6/11 would've passed.

In [14]:
propsal_choices = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Lido"].loc["0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Lido'][plutocracy_report_data['Lido']['proposal_id'] == '0xcbf534335fe07c046caa933e1623ac38bfb3d1890ab825264a0b47415cf7799b'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,"1. For 6 new oracles, 6/11 set","2. For 4 new oracles, 5/9 set",3. Against
Scores,96886.616711,5.272470e+07,4778.954352
Score Differeces,92934.922644,5.272312e+07,4535.772055


Here we see that whale participation was consistent for all choices for this proposal. In fact, ~99.982% of voting power allocated to this proposal came from whales that voted for each of the outcomes. The second outcome alone had ~99.980% of the voting power allocated to this proposal, so obviously its removal would make a huge impression on the outcome.

Indeed, the voting power would be comparitively low without whale intervention (if this were the case, only ~9277.7397 LDO would be allocated to this proposal). However the fact that after doing so additional oracles would still be implemented anyway, where the only other alternative would be to not implement any more oracles, shows that the community that participated in this proposal still saw it important to add more oracles to further promote decentralisation of oracles which [report chain state from the beacon chain to the execution layer](https://docs.lido.fi/guides/oracle-operator-manual) of ETH 2.0 as it pertains to the operation of Lido.

In conclusion of this analysis of this proposal, we can say that whales and average token holders were moving the same direction, but perhaps taking different paths. It will be interesting to see why whales opted to implement less oracles than what the majority of average token holders chose, which may require a more in depth report based on the [forum discussions](https://research.lido.fi/t/expansion-of-lidos-ethereum-oracle-set/2836) on the propoal.

Now we take a look at Frax.

In [15]:
score_diffrences_dfs["Frax"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x1272233285b3b4ce026173ec4e2a59d6c8daba3d64b41e67ba160cc526fc5c7d,weighted,"['For', 'Against']","[5786463.892845949, 0]",7350016.336582876,False,For
0x9cc997cf19f797919e40d87e17c7eed87aefa10279fbeef89761cb1717ac56b8,weighted,"['For', 'Against']","[3805819.97255508, 0]",6123819.651744431,False,For
0x02aa360090c273c1218b84fe4b25e2be1d6a0210be1419087832da3ac15c38eb,weighted,"['For', 'Against']","[3805825.454883847, 0]",6123455.989533540,False,For
0xe53269e838ab2ca0d8e09f8326d2a4ffb3425ecb1b39dfe89b72b7178b486614,weighted,"['For', 'Against']","[3805830.2519215182, 0]",6123331.650428076,False,For
0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a,weighted,"['For', 'Against']","[5792040.529852133, 0.0]",10207860.835706450,False,For


Whilst enourmous voting power was filtered out from all the proposals listed, none had their outcomes changed.

If we look at the proposal which attracted the most voting power, [this proposal](https://snapshot.org/#/frax.eth/proposal/0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a) to renew the grant for Flywheel for 2023 (in the amount of approx $214.2k, split 50/50 in FXS and FRAX) for the promotion of "the Frax ecosystem and producing high-qulity DeFi content", we see that just over 50% of the voting power was removed from this proposal after filtering out whales (approx 56.7%).

Though, because the opposition voting power for this proposal was a paltry 1E-5% of the vote, filtering out whales did not change the outcome. This clearly was not a controversial outcome to this proposal by number of voting participants (FXS token holders which participated in this propsal).

Indeed, Frax's [forum post](https://gov.frax.finance/t/fip-178-flywheel-grant-renewal-for-2023/2068) regarding this proposal had mostly positive things to say about Flywheel, which started out as a podcast promoting Frax in the wake of the collapse of Terra/Luna. Stating that since their last grant given to Flywheel in 2022, their social following has increased by 2770% on Twitter and 3140% on Youtube. They also serve as an educating branch for Frax, organizing meetups and also "attracted builders from across the ecosystem".

What's interesting about this proposal is that whale participation contributed to just over 56.7% of the voting power for this proposal. This is probably the most fairly represented outcome looked at so far, with the outcome being clearly popular with whales and the average voter.


In [23]:
propsal_choices = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Frax"].loc["0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Frax'][plutocracy_report_data['Frax']['proposal_id'] == '0xece8d5be8b180b54350c4bddee190e24e2849d233f8aac11e0ef0aa7d658ae2a'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,For,Against
Scores,1.020773e+07,131.88977
Score Differeces,5.792041e+06,0.00000


If we now look at Decentraland, we actually observe that there is quite a bad case of concentration of votes in the hands of whales.

In [18]:
score_diffrences_dfs["Decentraland"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x73fa3d2c5985fc10d792a48ba0c161812813bd2c56fa07c37520b45fb6d6e2ac,single-choice,"['yes', 'no']","[220232.192672885, 0]",228726.774457250,False,yes
0xdfc9d87313c293cbde71adbed9d585ccd79d28ae1dde555f674613920c8856c8,single-choice,"['yes', 'no']","[3358.9554568534227, 481138.6966000833]",485125.193938830,True,yes
0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5,single-choice,"['Yes', 'No', 'Invalid question/options']","[61658.53397015308, 1156691.3340859897, 0.0]",1219441.888908949,True,Yes
0xbf2891a1e4490c0ba14e3897c239de06483a2758dff2c8769c9f7ce9f4dfae2d,single-choice,"['yes', 'no']","[3416782.371943629, 0.0]",3433412.362236338,False,yes
0x6576a4e2a22df3660d64a5af2a0d6c4819aa86ec40d420f1f1a99359dbf174fc,single-choice,"['Unban', 'Ban', 'Invalid question/options']","[727947.7894103363, 0.0, 538722.9524735583]",1294131.760398563,False,Unban


For example, this proposal to set a duration period for the tenure of Decentraland DAO commitee members saw 99.9% of voting power being concentrated to whales, with 94.85% of propsal voting power being allocated to voting for the proposal to not pass.

In [25]:
propsal_choices = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Decentraland"].loc["0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Decentraland'][plutocracy_report_data['Decentraland']['proposal_id'] == '0x7f6fed8c7645d1b793526564104e4f79864a9e30ae284029f752b6297478b4f5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No,Invalid question/options
Scores,62474.116173,1.156968e+06,0.025206
Score Differeces,61658.533970,1.156691e+06,0.000000


Obviously a case of centralized power voting for their own self interests, its not suprising that if whales were factored out of the proposal (which would've given existing and future committee members set terms, making the roles more democratic) the proposal would've passed, albeit, with very low voting power in comparison.

Going back to DeFi, lets now look at Curve Finance

In [19]:
score_diffrences_dfs["Curve Finance"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5,single-choice,"['Yes', 'No']","[23462.994316173717, 511330.4724932205]",574553.874246682,True,Yes
0x43c689ef6b7b1772abc033cd87b7caa4159f0dc281f9a159e3930fb00b57d707,single-choice,"['Do Nothing', '2Pool + current 3Pool', '2Pool...","[0.0, 559204.7021513436, 2510541.702086272]",4272657.271177988,False,2Pool + current 3Pool + 4Pool
0xa11d4d1800b91746afe8ead922eb1d37225a32145adf30bbda0cbdba2ef97bd3,single-choice,"['Yes', 'No']","[26054297.07744241, 0]",26097304.209488451,False,Yes
0x026a69e5c9780479332d88fa9779a3cec5a9479d05cc4ca61b1fa9eb099caa73,single-choice,"['yes', 'no']","[1891.6266956075615, 0]",6893.077810757,False,yes


In [26]:
propsal_choices = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_choices']
proposal_score_diffrences = score_diffrences_dfs["Curve Finance"].loc["0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5"]["score_diffrences"]
proposal_scores = plutocracy_report_data['Curve Finance'][plutocracy_report_data['Curve Finance']['proposal_id'] == '0x0eb23ea0b877666ad3ddcd0d7da0114acdfe5ae6390b5628b7509f4338022db5'].iloc[0]['proposal_scores']

pd.DataFrame(
    {choice: [score, score_diff] for choice, score, score_diff in zip(eval(propsal_choices), eval(proposal_scores), proposal_score_diffrences)},
    index=["Scores", "Score Differeces"],
)

,Yes,No
Scores,45352.592228,529201.282019
Score Differeces,23462.994316,511330.472493


In [20]:
score_diffrences_dfs["Radicle"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0xf95b0e03c92f9f2df976cad6c5121383f4efc887162d0bf6b59cc550700bceec,single-choice,"['For', 'Against']","[2000000.0, 0]",4133689.798082654,False,For
0x646e92794f8ea0abb333ed264dc8d8a3df7ceae909186f88701b3237852e61fb,single-choice,"['Yes', 'No', 'Abstain']","[3730867.698436455, 0, 0]",6163639.615623794,False,Yes
bafkreia4tl25spzwtys4kfns3iiuj32n2wugyupa6adg6mlxwg77a3bzx4,single-choice,"['For', 'Make No Change', 'Abstain']","[2654186.698436455, 0, 0]",6922974.673405429,False,For
0x16400b2c86d6ebdea246372182f18a17394435d9e31f9749ebaad89e37f3eb25,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4971478.525081047,False,For
0x704025e577ff7e4e202985d342c30285c4d0d320a18c34b764aa9b4dd38fd8ec,basic,"['For', 'Against', 'Abstain']","[3090365.0190418432, 0, 0]",4946306.104718991,False,For


In [21]:
score_diffrences_dfs["Euler"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x3e7263667b5156ca065aa6809017134c73b903f09b8aa6b77b61b76cc0f09901,single-choice,"['Yes: deploy Euler to BNB Chain', 'No: make n...","[0.0, 16459.295824550856, 0.0]",37704.026691998,True,Abstain
0x8721e87559f34b71b5ffc51cb9717d90677259d7a2b3d1fd06524f86d3202745,single-choice,"['Delegate $SAFE to StableLab', 'Delegate $SAF...","[100000.0, 14950.714739293624, 0, 317969.84325...",488824.007486842,False,Abstain
0xe16545bf8aa67399720dba057cf9d23f1e73caa8dad8463fef783a64c89c1837,single-choice,"['YES - Implement changes', 'NO- Do not Implem...","[332920.55799487286, 0]",388710.341413462,False,YES - Implement changes
0x3635964ebd6535eb25ea522b8aa5981942c264249185506bb1bd9474b7c9ca72,single-choice,"['Yes - make rETH collateral', 'No - do not ma...","[332920.55799487286, 0.0]",397650.270054811,False,Yes - make rETH collateral
0x4129f05639d2ca2f76cba392a625fa2ebe9b32964d29e7e45dc831a8792d8af9,single-choice,"['YES - implement the changes ', 'NO -do not i...","[54944.92856057068, 0.0]",105724.703030176,False,YES - implement the changes


In [24]:
score_diffrences_dfs["JPEG’d"]

,type,choices,score_diffrences,total_vp,has_changed_outcome,changed_outcome_winner
Proposal ID,,,,,,
0x55723c4085277bf201baea0e0d298dfd255746efd194e25642cd976ecec3b1ba,single-choice,"['Yes approve new JPEG boost', 'No']","[1886964369.3630195, 0.0]",2501677551.837059975,False,Yes approve new JPEG boost
0xf1df891a0b7b080b7903386e47332c555d893cef5ccafc08f151a3d85b65e325,single-choice,"['Yes', 'No']","[1886964369.3630195, 0.0]",3291369012.976878166,False,Yes
0x43983b0760fb917c7cd0934da37788f60cd950ed275fc94b7bf0372a6858f872,single-choice,"['Yes, launch the index', 'No']","[0.0, 1120208158.6913748]",2026389221.211853027,False,No
0x0800198644ae03b79e74a37825b14e4e37be8289def68a073f01ecf829ce584a,single-choice,"['Yes implement tiered vaults', 'No']","[782222810.2662988, 0.0]",2367931826.085281849,False,Yes implement tiered vaults
0x7d80f7739cc1697b3af01ad58591cc6c2318b949440957965b9fbe16f5374077,single-choice,"['Yes - add support', 'No']","[1574931318.1311798, 0.0]",3337571044.563824177,False,Yes - add support
